In [2]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [4]:
transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

In [13]:
import shutil
dataset = ImageFolder('PlantVillage/', transform=transform)
class_names = dataset.classes



In [14]:
class_names

['Pepper__bell___Bacterial_spot',
 'Pepper__bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [15]:
train_size=int(0.8*len(dataset))
val_size=len(dataset)-train_size
train_ds,val_ds=random_split(dataset,[train_size,val_size])

In [16]:
train_loader=DataLoader(train_ds,batch_size=32,shuffle=True)
val_loader=DataLoader(val_ds,batch_size=32)

In [17]:
import torch.nn as nn
import torch.nn.functional as F

In [30]:
class PlantCNN(nn.Module):
    def __init__(self,num_classes):
        super(PlantCNN,self).__init__()
        self.conv1=nn.Conv2d(3,16,3)
        self.conv2=nn.Conv2d(16,32,3)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(32*30*30,128)
        self.fc2=nn.Linear(128,num_classes)
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,32*30*30)
        x=F.relu(self.fc1(x))
        return self.fc2(x)
    

In [31]:
model=PlantCNN(num_classes=len(class_names))

In [32]:
from torch.optim import Adam
import torch

In [34]:
device=torch.device('cude' if torch.cuda.is_available() else 'cpu')
model=model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=Adam(model.parameters(),lr=0.001)

In [35]:
for epoch in range(5):
    model.train()
    total_loss=0
    for images,labels in train_loader:
        images, labels=images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss:.4f}')

Epoch 1, Loss: 600.0746
Epoch 2, Loss: 268.0508
Epoch 3, Loss: 177.2047
Epoch 4, Loss: 128.1267
Epoch 5, Loss: 97.6800


In [39]:
from sklearn.metrics import classification_report
model.eval()
all_preds = []
all_labels = []

In [42]:
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())
print(classification_report(all_labels, all_preds, target_names=class_names))

                                             precision    recall  f1-score   support

              Pepper__bell___Bacterial_spot       0.84      0.90      0.87       206
                     Pepper__bell___healthy       0.97      0.92      0.94       308
                      Potato___Early_blight       0.91      0.93      0.92       209
                       Potato___Late_blight       0.80      0.84      0.82       209
                           Potato___healthy       0.95      0.67      0.78        30
                      Tomato_Bacterial_spot       0.80      0.96      0.87       390
                        Tomato_Early_blight       0.68      0.57      0.62       186
                         Tomato_Late_blight       0.87      0.79      0.83       395
                           Tomato_Leaf_Mold       0.86      0.83      0.84       187
                  Tomato_Septoria_leaf_spot       0.88      0.78      0.83       358
Tomato_Spider_mites_Two_spotted_spider_mite       0.85      0.90